In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report


In [2]:
# Load the dataset
df = pd.read_csv('train.csv')

df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation and special characters (keep basic words and spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Apply cleaning to comment_text
df['cleaned_comment_text'] = df['comment_text'].apply(clean_text)


In [4]:
# Features: the cleaned comment text
X = df['cleaned_comment_text']

# Labels: the 6 different toxic behaviors
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# Split the data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])


Training set size: 127656
Test set size: 31915


In [5]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit TF-IDF on the training set, then transform both training and test sets
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF matrix for training set:", X_train_tfidf.shape)
print("TF-IDF matrix for test set:", X_test_tfidf.shape)


TF-IDF matrix for training set: (127656, 10000)
TF-IDF matrix for test set: (31915, 10000)


In [6]:
# Create the Logistic Regression model
lr = LogisticRegression(solver='liblinear')  # 'liblinear' is good for small/medium datasets

# Wrap it in OneVsRestClassifier for multi-label classification
clf = OneVsRestClassifier(lr)

# Train (fit) the classifier on the TF-IDF features
clf.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_tfidf)


In [7]:
print(classification_report(y_test, y_pred, target_names=y.columns))


               precision    recall  f1-score   support

        toxic       0.91      0.61      0.73      3056
 severe_toxic       0.57      0.26      0.36       321
      obscene       0.92      0.63      0.74      1715
       threat       0.79      0.15      0.25        74
       insult       0.83      0.50      0.62      1614
identity_hate       0.73      0.15      0.25       294

    micro avg       0.88      0.55      0.67      7074
    macro avg       0.79      0.38      0.49      7074
 weighted avg       0.87      0.55      0.67      7074
  samples avg       0.06      0.05      0.05      7074



C:\Users\Pratik Singh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Pratik Singh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Pratik Singh\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
def interpret_prediction(prediction_row):
    if sum(prediction_row) == 0:
        return "Totally Fine"
    else:
        labels = []
        if prediction_row[0] == 1: labels.append("toxic")
        if prediction_row[1] == 1: labels.append("severe_toxic")
        if prediction_row[2] == 1: labels.append("obscene")
        if prediction_row[3] == 1: labels.append("threat")
        if prediction_row[4] == 1: labels.append("insult")
        if prediction_row[5] == 1: labels.append("identity_hate")
        return ", ".join(labels)

# Example: interpret the first 5 predictions
for i in range(5):
    print(f"Comment: {X_test.iloc[i]}")
    print(f"Prediction: {interpret_prediction(y_pred[i])}")
    print("-"*50)


Comment: geez are you forgetful  weve already discussed why marx  was  not an anarchist ie he wanted to use a state to mold his socialist man  ergo he is a statist  the opposite of an  anarchist  i know a guy who says that when he gets old and his teeth fall out hell quit eating meat  would you call him a vegetarian
Prediction: Totally Fine
--------------------------------------------------
Comment: carioca rfa 

thanks for your support on my request for adminship

the final outcome was 3141 so i am now an administrator if you have any comments or concerns on my actions as an administrator please let me know thank you
Prediction: Totally Fine
--------------------------------------------------
Comment: 

 birthday 

no worries its what i do enjoy ur daytalke 
Prediction: Totally Fine
--------------------------------------------------
Comment: pseudoscience category 

im assuming that this article is in the pseudoscience category because of its association with creationism  however there

In [9]:

test_comments = [
    "I love your work. It's amazing!",               
    "You are stupid.",                               
    "I will kill you.",                              
    "That is so disgusting and obscene!",            
    "Get out, you worthless scum.",                  
    "Go to hell, you piece of trash!",               
    "We don't want you around here!",                
    "If you don't do it, I'm going to find you.",
    "You are a worthless piece of garbage.",          
    "Hello, how are you today?"                       
]


In [10]:
# Clean the new comments
test_comments_cleaned = [clean_text(c) for c in test_comments]

# Convert them to TF-IDF features using the same fitted vectorizer
test_comments_tfidf = tfidf.transform(test_comments_cleaned)


In [11]:
# Predict labels
predictions = clf.predict(test_comments_tfidf)

# Print out the results
for comment, pred in zip(test_comments, predictions):
    print("Comment:", comment)
    print("Prediction:", interpret_prediction(pred))
    print("-"*50)


Comment: I love your work. It's amazing!
Prediction: Totally Fine
--------------------------------------------------
Comment: You are stupid.
Prediction: toxic, obscene, insult
--------------------------------------------------
Comment: I will kill you.
Prediction: toxic, threat
--------------------------------------------------
Comment: That is so disgusting and obscene!
Prediction: toxic
--------------------------------------------------
Comment: Get out, you worthless scum.
Prediction: toxic, insult
--------------------------------------------------
Comment: Go to hell, you piece of trash!
Prediction: toxic, insult
--------------------------------------------------
Comment: We don't want you around here!
Prediction: Totally Fine
--------------------------------------------------
Comment: If you don't do it, I'm going to find you.
Prediction: Totally Fine
--------------------------------------------------
Comment: You are a worthless piece of garbage.
Prediction: toxic, insult
------

In [13]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [14]:
import joblib

# Save the model to a file named "text_classification_model.pkl"
joblib.dump(clf, "text_classification_model.pkl")
print("Model saved as text_classification_model.pkl")


Model saved as text_classification_model.pkl


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)

# Save the vectorizer
import joblib
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [16]:
# Predict labels
predictions = clf.predict(test_comments_tfidf)

# Print out the results
for comment, pred in zip(test_comments, predictions):
    print("Comment:", comment)
    print("Prediction:", interpret_prediction(pred))
    print("-"*50)


Comment: I love your work. It's amazing!
Prediction: Totally Fine
--------------------------------------------------
Comment: You are stupid.
Prediction: toxic, obscene, insult
--------------------------------------------------
Comment: I will kill you.
Prediction: toxic, threat
--------------------------------------------------
Comment: That is so disgusting and obscene!
Prediction: toxic
--------------------------------------------------
Comment: Get out, you worthless scum.
Prediction: toxic, insult
--------------------------------------------------
Comment: Go to hell, you piece of trash!
Prediction: toxic, insult
--------------------------------------------------
Comment: We don't want you around here!
Prediction: Totally Fine
--------------------------------------------------
Comment: If you don't do it, I'm going to find you.
Prediction: Totally Fine
--------------------------------------------------
Comment: You are a worthless piece of garbage.
Prediction: toxic, insult
------